In [ ]:
import numpy as np
import pandas as pd
from collections import defaultdict
import copy

In [ ]:
#this file I sent to you via email

In [ ]:
df = pd.read_csv('2021_03_12_slate.csv')


In [ ]:
df

,PLAYER_NAME,POSITIONS,SALARY,FPTS,1_day_lag_season_averageFPTS
0,LeBron James,PG/SF,10400.0,39.25,51.027778
1,Wesley Matthews,SG/SF,3000.0,4.50,8.553030
2,Markieff Morris,PF,4300.0,22.25,10.641892
3,Dennis Schroder,PG/SG,6200.0,31.25,27.628788
4,Kentavious Caldwell-Pope,SG,3700.0,16.25,16.045455
...,...,...,...,...,...
169,Sterling Brown,SG/SF,4500.0,23.75,18.846774
170,Kevin Porter Jr.,SG/SF,3200.0,47.75,42.250000
171,Anthony Lamb,PF,3300.0,14.50,4.750000
172,Jae'Sean Tate,SF/PF,5800.0,17.50,21.792857


In [ ]:
def modify_position_eligibility(row):
    try:
        position_list = row.split('/')
        for val in position_list:
            if 'F' in val and 'F' not in position_list:
                position_list.append('F')
            if 'G' in val and 'G' not in position_list:
                position_list.append('G')
        position_list.append('UTIL')
        return '/'.join(position_list)
    except:
        position_list = [row]
        if 'F' in row:
            position_list.append('F')
        if 'G' in row:
            position_list.append('G')
        position_list.append('UTIL')
        return '/'.join(position_list)

In [ ]:
positions_to_fill = ['PG','SG','SF','PF','C','F','G','UTIL']
df['POSITIONS'] = df['POSITIONS'].apply(modify_position_eligibility)

In [ ]:
#As you can see below, every player has multiple position eligibilities

In [ ]:
#best lineup without budget constraint would be:

#Nikola Jokic - C
#Jimmy Butler - SF
#Donovan Mitchell - PG
#Domantas Sabonis - PF
#Zach LaVine - SG
#Malcom Brogdon - G
#Kevin Porter Jr. - F
#Brandon Ingram - UTIL

#Total score 412.5
#Total salary spent 67900

In [ ]:
df.sort_values('FPTS',ascending=False).head(50)

,PLAYER_NAME,POSITIONS,SALARY,FPTS,1_day_lag_season_averageFPTS
68,Nikola Jokic,C/UTIL,10900.0,62.75,59.222222
15,Jimmy Butler,SF/F/UTIL,9200.0,52.75,48.068182
48,Donovan Mitchell,PG/SG/G/UTIL,8800.0,52.25,41.448529
158,Domantas Sabonis,PF/C/F/UTIL,9900.0,51.50,47.860294
83,Zach LaVine,SG/G/UTIL,9600.0,51.00,45.435714
159,Malcolm Brogdon,PG/SG/G/UTIL,8000.0,48.25,39.441176
170,Kevin Porter Jr.,SG/SF/G/F/UTIL,3200.0,47.75,42.250000
129,Brandon Ingram,SF/PF/F/UTIL,8300.0,46.25,40.162162
9,Kyle Kuzma,SF/PF/F/UTIL,5900.0,45.75,24.284722
93,Russell Westbrook,PG/G/UTIL,10200.0,45.25,49.928571


In [ ]:
#purpose of this function is that some players have eligibility for more than one position as you can see in the dataframe above

def check_valid_position(row, positions=[]):
    position_list = row.split('/')
    for val in position_list:
        if val in positions:
            return True
    return False

def get_legal_actions(dataframe, empty_positions, players_lineup, budget_left):

    def filter_available(row):
        return check_valid_position(row, positions=empty_positions)

    player_is_available = dataframe['POSITIONS'].apply(filter_available)
    players_not_in_lineup = np.logical_not(dataframe['PLAYER_NAME'].isin(players_lineup))
    budget_is_enough = np.array(dataframe['SALARY']) <= budget_left
    player_is_valid = np.logical_and(player_is_available, players_not_in_lineup, budget_is_enough)
    return list(dataframe[player_is_valid]['PLAYER_NAME'].unique())


In [ ]:
def UCB(values, visits, c=np.sqrt(2), unexplored_value=np.inf):
    exploration = c * np.sqrt( np.log(np.sum(visits)) / visits )
    exploration = np.where(visits == 0, unexplored_value, exploration)
    return values + exploration


In [ ]:
def UCB_tuned(values, visits, c=np.sqrt(2), unexplored_value=np.inf):
    second_term = min(1/4, (np.var(values) + (2*np.log(np.sum(visits))/visits)))
    exploration = np.sqrt( (np.log(np.sum(visits)) / visits) * second_term)
    # exploration = c * np.sqrt( np.log(np.sum(visits)) / visits )
    exploration = np.where(visits == 0, unexplored_value, exploration)
    return values + exploration


In [ ]:
class MCTSNode():

    def __init__(self, dataframe, lineup:list,
                 empty_positions:list, budget_left:float, nodes:list):
        self.dataframe = dataframe
        self.lineup = lineup
        self.lineup.sort()
        self.budget_left = budget_left
        self.empty_positions = empty_positions

        self.nodes = nodes
        self.legal_actions = get_legal_actions(dataframe, empty_positions, self.players_lineup, self.budget_left)

        self.children = [None for action in self.legal_actions]
        self.children_visits = np.array([0 for action in self.legal_actions])
        self.children_values = np.array([0 for action in self.legal_actions])
        self.children_cost = np.array([
            self.dataframe[self.dataframe['PLAYER_NAME']==action]['SALARY'].values[0]
            for action in self.legal_actions
        ])

        self.value = None

    def build_children(self, action:str):
        new_empty_positions = copy.copy(self.empty_positions)
        position = self.dataframe[self.dataframe['PLAYER_NAME']==action]['POSITIONS'].values[0]
        if '/' in position:
            positions = position.split('/')
            for position_possibility in positions:
                if position_possibility in new_empty_positions:
                    position = position_possibility
                    break

        new_empty_positions.remove(position)

        new_lineup = self.lineup + [(position, action)]
        new_lineup.sort()
        coresponding_nodes = list(filter(lambda n: n == new_lineup, self.nodes))
        if len(coresponding_nodes) > 0:
            return coresponding_nodes[0]

        cost = self.dataframe[self.dataframe['PLAYER_NAME']==action]['SALARY'].values[0]
        new_budget_left = self.budget_left - cost

        new_node = MCTSNode(self.dataframe, new_lineup, new_empty_positions,
                            new_budget_left, nodes=self.nodes)
        self.nodes.append(new_node)
        return new_node

    def get_child_by_id(self, child_id):
        child = self.children[child_id]
        if child is None:
            action = self.legal_actions[child_id]
            child = self.build_children(action)
            self.children[child_id] = child
        return child

    @property
    def players_lineup(self):
        return [player for position, player in self.lineup]

    @property
    def is_leaf(self) -> bool:
        return len(self.legal_actions) == 0 or np.any(self.children_visits <= 0)

    @property
    def is_terminal(self) -> bool:
        return len(self.legal_actions) == 0 #TODO Or no budget left

    def __repr__(self):
        return f"Node({self.budget_left}$, {self.lineup})"

    def __eq__(self, other):
        if isinstance(other, list):
            return self.lineup == other
        return self.lineup == other.lineup

class MCTSTree():

    def __init__(self, dataframe, empty_positions, budget, exploration=1.):
        self.nodes = []
        self.root = MCTSNode(dataframe, [], empty_positions, budget, nodes=self.nodes)
        self.nodes.append(self.root)
        self.exploration = exploration
        self.best_node = None

    def run(self, n_simulations=1):
        """ Run the MCTS search for a number of simulations """
        for sim in range(n_simulations):
            start_node, path, actions_ids = self.select()
            sim_path, sim_actions_ids = self.simulation(start_node)
            reward = self.evaluate(sim_path[-1])
            self.backpropagate(reward, path[:-1] + sim_path, actions_ids + sim_actions_ids)
            if sim == 0 or (sim+1)%(n_simulations//10)==0:
                print(f"{sim+1}/{n_simulations} - {len(self.nodes)} nodes")

    def select(self) -> MCTSNode:
        """ Select a leaf node to start from
        
        Return:
            Node choosen
        
        """
        node = self.root
        path = [self.root]
        actions_ids = []

        while not node.is_leaf:
            selection_criterion = UCB(
                node.children_values, node.children_visits, self.exploration)
            best_child_id = np.argmax(selection_criterion)
            node = node.get_child_by_id(best_child_id)
            path.append(node)
            actions_ids.append(best_child_id)

        return node, path, actions_ids

    def simulation(self, start_node:MCTSNode):
        """ Choose a full lineup building nodes on the way
        
        Return:
            Path of nodes used in simulation
            Action IDs used at each node
        
        """

        node = start_node
        path = [start_node]
        actions_ids = []

        while not node.is_terminal:
            #TODO priority to non-built children ?
            action_ids = range(len(node.legal_actions))

            # # print(node, node.children_visits)
            # if np.any(node.children_visits == 0):
            #     action_id = np.argmax(node.children_visits == 0 * node.children_cost)
            # else:
            action_id = np.random.choice(action_ids)

            node = node.get_child_by_id(action_id)
            path.append(node)
            actions_ids.append(action_id)

        return path, actions_ids

    def evaluate(self, node) -> float:
        """ Evaluate a completed lineup
        
        Return:
            Value of the node
        
        """
        if not node.is_terminal:
            raise ValueError("Non-terminal nodes should never be evaluated")
        if node.value is None:
            node.value = df[df['PLAYER_NAME'].isin(node.players_lineup)]['FPTS'].sum()

        if self.best_node is None or self.best_node.value < node.value:
            self.best_node = node
            print(f'New best node: {node} with value {node.value}')

        return node.value

    def backpropagate(self, reward, path, actions_ids):
        """ Backpropagate a reward through a trajectory """
        for node, action_id in zip(path[:-1], actions_ids):
            node.children_visits[action_id] += 1
            n = node.children_visits[action_id]
            q = node.children_values[action_id]
            node.children_values[action_id] += (reward - q) / n


In [ ]:
positions_to_fill

['PG', 'SG', 'SF', 'PF', 'C', 'F', 'G', 'UTIL']

In [ ]:
#For some reason here, despite the fact that we indicated there are 8 positions to fill, it only ever creates lineups with 5 players
#I did a bunch of print statements locally and the source of the error seems to at the 'break' statement in the build_children function
#I put a commented out print statement there to show you what I mean. If you uncomment it and run you'll see that it never actually reaches 'G', 'F', or 'UTIL'

In [ ]:
tree = MCTSTree(dataframe=df.copy(deep=True), empty_positions=positions_to_fill, budget=50000, exploration=1)
tree.run(1000)

# for node in tree.nodes:
#     print(node.lineup, node.children_values, node.children_visits)

New best node: Node(14300.0$, [('C', 'Chris Silva'), ('F', 'Luka Samanic'), ('G', 'Tyus Jones'), ('PF', 'Doug McDermott'), ('PG', 'Collin Sexton'), ('SF', "Jae'Sean Tate"), ('SG', 'Raul Neto'), ('UTIL', 'KZ Okpala')]) with value 129.75
1/1000 - 9 nodes
New best node: Node(1500.0$, [('C', 'Montrezl Harrell'), ('F', 'Lauri Markkanen'), ('G', 'Damyean Dotson'), ('PF', 'Tobias Harris'), ('PG', 'Collin Sexton'), ('SF', 'Luka Samanic'), ('SG', 'Zach LaVine'), ('UTIL', 'Ben McLemore')]) with value 213.5
New best node: Node(3100.0$, [('C', 'Nikola Jokic'), ('F', "Royce O'Neale"), ('G', 'Seth Curry'), ('PF', 'Mike Scott'), ('PG', "De'Anthony Melton"), ('SF', 'Doug McDermott'), ('SG', 'Furkan Korkmaz'), ('UTIL', 'Jarrett Allen')]) with value 242.0
New best node: Node(600.0$, [('C', 'Wendell Carter Jr.'), ('F', 'Georges Niang'), ('G', 'Zach LaVine'), ('PF', 'Lauri Markkanen'), ('PG', 'Nickeil Alexander-Walker'), ('SF', 'Jimmy Butler'), ('SG', 'Joe Ingles'), ('UTIL', 'Montrezl Harrell')]) with val

In [ ]:
print(tree.root.children_visits)

[ 6  7  2  5  5  5  4  7  7  6  5  3 10  4  4  8  5  3  5  8  8  8  5  8
  5  5  3  4  6 10  4  7  3  5  8  3 11  4  3  6  4  4  0  4  6  6  4  3
  4  7  7  8  3  7  5  8  3  5  2 12  8  8  4  5  3  5  6  7  9  3  6  7
  6  5  3  7  8  6  9  4  3  7  8  6  7  5  3  3  8  5  6  5  5  4  5  8
  6  2  4  8  4  7  4  7  7 11  2  9  5  6  7  7  8  3  5  3  7  6  3  7
  3  9  4  5  8  7  6  7 12  5  4  4  2  3  5  9 11  8  5  7  9  8  5  3
  6  5  5  5  8 11  8  7  6  4  6  7  5  5  4  6  3  7  7  6  3 10  4  3
  6  5  7  6  8  6]


In [ ]:
print(tree.root.legal_actions)

['LeBron James', 'Wesley Matthews', 'Markieff Morris', 'Dennis Schroder', 'Kentavious Caldwell-Pope', 'Montrezl Harrell', 'Damian Jones', 'Alex Caruso', 'Alfonzo McKinnie', 'Kyle Kuzma', 'Talen Horton-Tucker', 'Devontae Cacok', 'Udonis Haslem', 'Andre Iguodala', 'Goran Dragic', 'Jimmy Butler', 'Maurice Harkless', 'Kelly Olynyk', 'Duncan Robinson', 'Kendrick Nunn', 'Gabe Vincent', 'Max Strus', 'Tyler Herro', 'KZ Okpala', 'Chris Silva', 'Precious Achiuwa', 'Dwight Howard', 'Danny Green', 'Tobias Harris', 'Mike Scott', 'Seth Curry', 'Joel Embiid', 'Furkan Korkmaz', 'Terrance Ferguson', 'Tony Bradley', 'Shake Milton', 'Matisse Thybulle', 'Vincent Poirier', 'Tyrese Maxey', 'Ersan Ilyasova', 'Mike Conley', 'Derrick Favors', 'Bojan Bogdanovic', 'Rudy Gobert', 'Jordan Clarkson', 'Joe Ingles', "Royce O'Neale", 'Georges Niang', 'Donovan Mitchell', 'Miye Oni', 'Jarrell Brantley', 'Juwan Morgan', 'Trent Forrest', 'Rudy Gay', 'Patty Mills', 'Trey Lyles', 'Dejounte Murray', 'Jakob Poeltl', 'Derrick 

In [ ]:
list(zip(tree.root.children_visits,tree.root.legal_actions))

[(6, 'LeBron James'),
 (7, 'Wesley Matthews'),
 (2, 'Markieff Morris'),
 (5, 'Dennis Schroder'),
 (5, 'Kentavious Caldwell-Pope'),
 (5, 'Montrezl Harrell'),
 (4, 'Damian Jones'),
 (7, 'Alex Caruso'),
 (7, 'Alfonzo McKinnie'),
 (6, 'Kyle Kuzma'),
 (5, 'Talen Horton-Tucker'),
 (3, 'Devontae Cacok'),
 (10, 'Udonis Haslem'),
 (4, 'Andre Iguodala'),
 (4, 'Goran Dragic'),
 (8, 'Jimmy Butler'),
 (5, 'Maurice Harkless'),
 (3, 'Kelly Olynyk'),
 (5, 'Duncan Robinson'),
 (8, 'Kendrick Nunn'),
 (8, 'Gabe Vincent'),
 (8, 'Max Strus'),
 (5, 'Tyler Herro'),
 (8, 'KZ Okpala'),
 (5, 'Chris Silva'),
 (5, 'Precious Achiuwa'),
 (3, 'Dwight Howard'),
 (4, 'Danny Green'),
 (6, 'Tobias Harris'),
 (10, 'Mike Scott'),
 (4, 'Seth Curry'),
 (7, 'Joel Embiid'),
 (3, 'Furkan Korkmaz'),
 (5, 'Terrance Ferguson'),
 (8, 'Tony Bradley'),
 (3, 'Shake Milton'),
 (11, 'Matisse Thybulle'),
 (4, 'Vincent Poirier'),
 (3, 'Tyrese Maxey'),
 (6, 'Ersan Ilyasova'),
 (4, 'Mike Conley'),
 (4, 'Derrick Favors'),
 (0, 'Bojan Bogdan